Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using DynamicHMCModels, Optim

### snippet 10.4

In [2]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[!, :pulled_left] = convert(Array{Int64}, df[!, :pulled_left])
df[!, :prosoc_left] = convert(Array{Int64}, df[!, :prosoc_left])
first(df, 5)

struct m_10_02d{TY <: AbstractVector, TX <: AbstractMatrix}
    "Observations."
    y::TY
    "Covariates"
    X::TX
    "Number of observations"
    N::Int
end

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::m_10_02d)(θ)
    @unpack y, X, N = problem   # extract the data
    @unpack β = trans(θ)  # works on the named tuple too
    ll = 0.0
    ll += sum(logpdf.(Normal(0, 10), β)) # a & bp
    ll += sum([loglikelihood(Binomial(1, logistic(dot(X[i, :], β))), [y[i]]) for i in 1:N])
    ll
end

Instantiate the model with data and inits.

In [4]:
N = size(df, 1)
X = hcat(ones(Int64, N), df[!, :prosoc_left]);
y = df[!, :pulled_left]
p = m_10_02d(y, X, N);

Function convert from a single vector of parms to parks NamedTuple

In [5]:
trans = as( (β = as(Array, size(p.X, 2)), ));

γ =  (β = [0.5, 1.0],)
θ = inverse(trans, γ)
p(θ)

-371.672843317438

Maximum_a_postrior

In [6]:
x0 = θ;
lower = [-1.0, -1.0];
upper = [1.0, 2.0];
ll(x) = -p(x)

inner_optimizer = GradientDescent()

optimize(ll, lower, upper, x0, Fminbox(inner_optimizer))

 * Status: success

 * Candidate solution
    Minimizer: [4.77e-02, 5.57e-01]
    Minimum:   3.446925e+02

 * Found with
    Algorithm:     Fminbox with Gradient Descent
    Initial Point: [5.00e-01, 1.00e+00]

 * Convergence measures
    |x - x'|               = 2.99e-10 ≰ 0.0e+00
    |x - x'|/|x'|          = 5.35e-10 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 9.39e-09 ≤ 1.0e-08

 * Work counters
    Iterations:    4
    f(x) calls:    532
    ∇f(x) calls:   532


Write a function to return properly dimensioned transformation.

In [7]:
problem_transformation(p::m_10_02d) =
  as( Vector, size(p.X, 2)) #    as( (β = as(Array, size(p.X, 2)), ) )

problem_transformation (generic function with 1 method)

Wrap the problem with a transformation, then use Flux for the gradient.

In [8]:
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Tune and sample.

In [9]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0017 s/step ...done
MCMC, adapting ϵ (25 steps)
0.00085 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0056 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0008 s/step ...done
MCMC, adapting ϵ (200 steps)
0.0016 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00066 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00076 s/step ...done
MCMC (1000 steps)
0.00065 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [10]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{Array{Float64,1},1}:
 [0.11799992973633616, 0.4886124331706821] 
 [-0.04089055969772584, 0.6567925561956877]
 [0.14030934626701774, 0.49201337522247607]
 [0.1236926146853002, 0.5177165318493312]  
 [-0.04854576792704324, 0.6769422645393532]

Extract the parameter posterior means: `β`,

In [11]:
posterior_a = mean(first, posterior)
posterior_bp = mean(last, posterior)

0.5515292673939926

Effective sample sizes (of untransformed draws)

In [12]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×2 Array{Float64,2}:
 1000.0  1000.0

NUTS-specific statistics

In [13]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.91, min/25%/median/75%/max: 0.29 0.86 0.95 1.0 1.0
  termination: AdjacentTurn => 27% DoubledTurn => 73%
  depth: 1 => 21% 2 => 77% 3 => 2% 4 => 0%


CmdStan result

In [14]:
m_10_2s_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
      Mean        SD       Naive SE       MCSE      ESS
 a 0.05103234 0.12579086 0.0019889282 0.0035186307 1000
bp 0.55711212 0.18074275 0.0028577937 0.0040160451 1000

Quantiles:
       2.5%        25.0%       50.0%      75.0%      97.5%
 a -0.19755400 -0.029431425 0.05024655 0.12978825 0.30087758
bp  0.20803447  0.433720250 0.55340400 0.67960975 0.91466915
";

Extract the parameter posterior means: `β`,

In [15]:
[posterior_a, posterior_bp]

2-element Array{Float64,1}:
 0.057657936214358316
 0.5515292673939926  

End of `10/m10.02d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*